# **Lab 6: Creando Reglas para Nuestras Instancias (Con Instance Connect)**

Este laboratorio te guiará en la creación y ajuste de **grupos de seguridad (Security Groups)**, la conexión a una instancia EC2 mediante **AWS Instance Connect** y la verificación de cómo las reglas de seguridad afectan la conectividad y el tráfico de red entre instancias públicas y privadas.

---

**Objetivos del Laboratorio**

1. **Configurar grupos de seguridad con reglas específicas:** Crear un grupo de seguridad para una instancia pública con acceso SSH y HTTP desde Internet, y otro grupo de seguridad para una instancia privada que solo permita tráfico SSH desde la instancia pública.
2. **Conectar a una instancia usando Instance Connect:** Acceder a la instancia pública sin necesidad de claves en tu máquina local, utilizando Instance Connect directamente desde la consola de AWS.
3. **Verificar el impacto de las reglas de seguridad:** Probar el acceso a la instancia privada desde la instancia pública, así como la disponibilidad de HTTP en la instancia pública, y observar cómo las reglas definidas afectan el tráfico.

---

**Instrucciones Detalladas**

**1. Crear Grupos de Seguridad**

1. En la consola de AWS, ve a **EC2**.
2. Selecciona **Security Groups** en el menú lateral.
3. Haz clic en **Create Security Group**.

**Grupo de Seguridad: `Public-SG`**

- Descripción: Permitir tráfico SSH y HTTP desde Internet.
- Reglas de entrada:
    - SSH (TCP/22) desde `0.0.0.0/0` (para usar Instance Connect).
    - HTTP (TCP/80) desde `0.0.0.0/0` (acceso web público).
- Reglas de salida: dejar la regla predeterminada (todo tráfico saliente permitido).

Crea el grupo.

**Grupo de Seguridad: `Private-SG`**

- Descripción: Permitir tráfico SSH desde la subred pública.
- Reglas de entrada:
    - SSH (TCP/22) desde el grupo de seguridad `Public-SG` (referencia cruzada).
- Reglas de salida: dejar la regla predeterminada.

Crea el grupo.

---

**2. Lanzar Instancias EC2**

1. **Instancia Pública**:
    - Nombre: `Public-Instance`
    - AMI: Amazon Linux 2
    - Tipo: t2.micro
    - Red: VPC y Subred pública previamente creadas en ejercicios anteriores.
    - Grupo de seguridad: `Public-SG`
    - Par de claves: No necesario para Instance Connect.
2. **Instancia Privada**:
    - Nombre: `Private-Instance`
    - AMI: Amazon Linux 2
    - Tipo: t2.micro
    - Red: VPC y Subred privada
    - Grupo de seguridad: `Private-SG`

Haz clic en **Launch Instances**.

---

**3. Conectar a la Instancia Pública con Instance Connect**

1. En la consola de EC2, selecciona la `Public-Instance`.
2. Haz clic en **Connect**.
3. Selecciona la pestaña **EC2 Instance Connect**.
4. Haz clic en **Connect**.
5. Ahora estás dentro de la instancia pública vía navegador, sin usar claves locales.

---

**4. Configurar un Servidor HTTP en la Instancia Pública**

1. Dentro de la `Public-Instance`:
    
    ```bash
    
    sudo yum install -y httpd
    sudo systemctl start httpd
    echo "Servidor HTTP de la Public-Instance" | sudo tee /var/www/html/index.html
    
    ```
    

Abre el navegador local en tu equipo y accede a:

```vbnet

http://<Public-Instance-Public-IP>

```

Deberías ver el mensaje: "Servidor HTTP de la Public-Instance".

---

**5. Conectarte a la Instancia Privada Desde la Instancia Pública**

1. Desde la terminal de la `Public-Instance`, ejecuta:
    
    ```bash
    
    ssh ec2-user@<Private-Instance-Private-IP>
    
    ```
    
2. Si la conexión es exitosa, significa que las reglas del `Private-SG` están funcionando, permitiendo SSH solo desde la instancia pública.
3. En la `Private-Instance`, verifica conectividad saliente:
    
    ```bash
    ping -c 4 google.com
    curl http://example.com
    
    ```
    
    Deberías tener acceso a Internet (asumiendo una NAT Gateway configurada en módulos anteriores).
    

---

**6. Ajustar Reglas y Verificar**

- Elimina la regla HTTP (puerto 80) en `Public-SG`.
- Intenta acceder nuevamente vía web a la `Public-Instance`. El acceso HTTP debería fallar.
- Cambia la regla SSH de `Public-SG` para que solo permita tu IP (My IP). Verifica que Instance Connect sigue funcionando.

---

**Resultados Esperados**

- Conectividad SSH a la instancia pública mediante Instance Connect.
- Acceso HTTP a la instancia pública controlado por las reglas del grupo de seguridad.
- Capacidad de conectarte desde la instancia pública a la privada, validando las referencias cruzadas de grupos de seguridad.
- Observación de cómo la eliminación o restricción de reglas afecta inmediatamente el tráfico entrante/saliente.

---

**Conclusiones**

En este laboratorio has:

- Aprendido a crear y configurar grupos de seguridad en AWS, estableciendo reglas de entrada y salida para controlar el tráfico.
- Utilizado AWS Instance Connect para acceder a la instancia sin configurar claves SSH a nivel local.
- Verificado cómo las reglas de seguridad influyen directamente en la conectividad entre instancias y hacia Internet, lo que te permite entender cómo asegurar entornos de AWS de manera flexible.
- Experimentado con la modificación de reglas para ver su efecto inmediato en el tráfico, lo que facilita el diseño de arquitecturas seguras y adaptables.

Este conocimiento es esencial para asegurar y administrar tus entornos en la nube de manera eficiente y segura.